In [25]:
import pandas as pd
import csv
import pickle
import numpy as np
from IPython.display import display
import multiprocessing
from multiprocessing import Pool
import math
from tqdm import tqdm
import time
import traceback
from sklearn.externals import joblib
import xgboost as xgb
import json
import time

In [26]:
path = "/Users/jay/MSC_WSBDA/MSc_Thesis/Msc_project/Data/"

In [72]:
event = 'givenchy'
start_hour = 15

In [73]:
def load_pickle_file(pickled_file):
    print(f'Loading data file from {pickled_file}')
    infile = open(pickled_file,'rb')
    unpickled_file = pickle.load(infile)
    print(f'Loaded {len(unpickled_file)} entries')
    infile.close()
    return unpickled_file
          
    
def save_pickle_file(path, data):
    print('Dumping data to path {}'.format(path))
    with open(path, 'wb') as file:
        pickle.dump(data, file)
    print('Finished dumping data to path {}'.format(path))


def mean(numbers):
    return float(sum(numbers)) / max(len(numbers), 1)


def safe_division(x, y):
    if y == 0:
        return 0
    else:
        return x/y

In [74]:
users = load_pickle_file(path+event+"_users.dat")
users.reset_index(drop =True , inplace =True)
simulation_result = pd.read_csv(path + 'Results/different_hours_training_model/simulation_result_'+str(start_hour)+'hrs_'+str(start_hour)+'hrs_model.csv')
#network_simulation.reset_index(drop =True , inplace =True)

Loading data file from /Users/jay/MSC_WSBDA/MSc_Thesis/Msc_project/Data/givenchy_users.dat
Loaded 5973 entries


In [75]:
simulation_comparison = pd.concat([simulation_result['time_lapsed'],users['time_lapsed']],axis = 1)
column_names = ['nw_time_lapsed','usr_time_lapsed']
simulation_comparison.columns = column_names

In [76]:
simulation_comparison_after_6hrs = simulation_comparison[simulation_comparison['usr_time_lapsed'] > (start_hour * 60)]

In [77]:
simulation_comparison_after_6hrs['both_infected'] = simulation_comparison_after_6hrs.apply(lambda x: 1 if (np.isnan(x['nw_time_lapsed']) == False)
                                                                          & (np.isnan(x['usr_time_lapsed']) == False)
                                                                     else 0,axis = 1)



/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [78]:
true_positive = simulation_comparison_after_6hrs['both_infected'].sum()
total_predicted_positive = len(simulation_comparison_after_6hrs[np.isnan(simulation_comparison_after_6hrs['nw_time_lapsed']) == False])
precision = true_positive/total_predicted_positive
total_actual_positive = len(simulation_comparison_after_6hrs[np.isnan(simulation_comparison_after_6hrs['usr_time_lapsed']) == False])
recall = true_positive/total_actual_positive
f1_score = (2*(precision * recall))/(precision + recall)


In [79]:
accuracy = {
'simulation_start_time':[],
'precision':[],
'recall':[],
'f1_score':[]
}
accuracy['simulation_start_time'].append(start_hour) 
accuracy['precision'].append(precision) 
accuracy['recall'].append(recall)
accuracy['f1_score'].append(f1_score) 
result = pd.DataFrame(accuracy) 
result.to_csv(path+'Results/after_time/simulation_accuracy_'+str(start_hour)+'hrs_'+str(start_hour)+'hrs_model_total_number_after_time.csv',index = False)